
- Stanford corenlp
   - tokenize and tag POS
- nltk
   - lemmatize
- replace nouns with meta
- replace quantities with meta


In [27]:

import csv
import json

import stanza
import nltk
from nltk.corpus import stopwords
from stanza import Pipeline

stanza.download('en')
nltk.download('stopwords')

2023-01-07 22:34:32 INFO: Downloading default packages for language: en (English) ...
2023-01-07 22:34:33 INFO: File exists: /home/jawad/stanza_resources/en/default.zip
2023-01-07 22:34:38 INFO: Finished downloading models and saved to /home/jawad/stanza_resources.
[nltk_data] Downloading package stopwords to /home/jawad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
nltk_sw = stopwords.words('english')
nltk_sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [29]:
statement = "Lauren wants to mix 5 liters of 7 % milk with skim - milk ( 0 % fat ) to produce a mixture of 2.9787 % milk . How much skim - milk should Lauren add ?"

In [30]:
nlp = Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')
doc = nlp(statement)

2023-01-07 22:34:48 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-07 22:34:48 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-01-07 22:34:49 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2023-01-07 22:34:49 INFO: Use device: gpu
2023-01-07 22:34:49 INFO: Loading: tokenize
2023-01-07 22:34:49 INFO: Loading: pos
2023-01-07 22:34:49 INFO: Loading: lemma
2023-01-07 22:34:49 INFO: Done loading processors!


In [40]:
def map_tokens(x) -> str:
    x = x.to_dict()
    if x['upos'] == 'PROPN':
        return '[#]'
    elif x['upos'] == 'NUM':
        return '[#]'
    else:
        return x['lemma']

def flt_tokens(x) -> bool:
    x = x.to_dict()
    return x['lemma'] not in nltk_sw

def norm_snt(i) -> list:
    return [w for w in map(map_tokens, filter(flt_tokens, i))]


In [42]:
ns = norm_snt(doc.iter_words())
ns

['[#]',
 'want',
 'mix',
 '[#]',
 'liter',
 '[#]',
 '%',
 'milk',
 'skim',
 '-',
 'milk',
 '(',
 '[#]',
 '%',
 'fat',
 ')',
 'produce',
 'mixture',
 '[#]',
 '%',
 'milk',
 '.',
 'much',
 'skim',
 '-',
 'milk',
 '[#]',
 'add',
 '?']

In [46]:

data = csv.reader(open('mawpscopy.csv'))
next(data)


['id', 'original_text', 'segmented_text', 'equation', 'ans']

In [47]:
eqn_types = {}

for row in data:
    text = row[1]
    eqn = row[3]
    pe = ''.join(filter(lambda x: x in '+-/*()', eqn))
    if pe not in eqn_types:
        eqn_types[pe] = [text]
    else:
        eqn_types[pe].append(text)

eqn_types    

{'*': ['Bryan took a look at his books as well . If Bryan has 56 books in each of his 9 bookshelves , how many books does he have in total ?',
  'Sara has saved 11 quarters from washing cars .   How many cents does Sara have ?',
  'Mrs. Hilt measured the distance from her desk to the water fountain . It was 30 feet . How many feet will Mrs. Hilt walk on her trips to the fountain if she goes to the water fountain 4 times today ?',
  'Sally saw 1 dozen birds in a tree . How many birds did Sally see ?',
  'Sally has 6 blue balloons . Fred has 3 times more   blue balloons than Sally . How many blue balloons does Fred have now ?',
  'Nancy has 7 black balloons . Mary has 4 times more   black balloons than Nancy . How many black balloons does Mary have now ?',
  'Mrs. Hilt saw 144 bees in the hive . The next day she saw 3 times that many . How many bees did she see on the second day ?',
  'Sara goes fishing with Melanie . Sara catches 5 trout . Melanie catches 2 times as many trout as Sara .

In [61]:
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

lds = []
n_eqn_types = {}

In [63]:
for k in eqn_types:
    n_eqn_types[k] = list(map(nlp, eqn_types[k]))
    print(n_eqn_types[k])

 60%|██████    | 52/86 [02:34<01:40,  2.96s/it]


[[
  [
    {
      "id": 1,
      "text": "Bryan",
      "lemma": "Bryan",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "start_char": 0,
      "end_char": 5
    },
    {
      "id": 2,
      "text": "took",
      "lemma": "take",
      "upos": "VERB",
      "xpos": "VBD",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "start_char": 6,
      "end_char": 10
    },
    {
      "id": 3,
      "text": "a",
      "lemma": "a",
      "upos": "DET",
      "xpos": "DT",
      "feats": "Definite=Ind|PronType=Art",
      "start_char": 11,
      "end_char": 12
    },
    {
      "id": 4,
      "text": "look",
      "lemma": "look",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Number=Sing",
      "start_char": 13,
      "end_char": 17
    },
    {
      "id": 5,
      "text": "at",
      "lemma": "at",
      "upos": "ADP",
      "xpos": "IN",
      "start_char": 18,
      "end_char": 20
    },
    {
      "id": 6,
      

KeyboardInterrupt: 